# Camera Calibration and Distorsion Correction

In [1]:
import numpy as np
import cv2
import glob
import pickle
from tqdm import tqdm

In [3]:
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9, 0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('./camera_cal/calibration*.jpg')

def camera_calibration(gray_image_data):
    images = np.copy(gray_image_data)
    
    # Step through the list and search for chessboard corners
    for idx, fname in tqdm(enumerate(images)):
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (9,6), None)

        # If found, add object points, image points
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)

            # Draw and display the corners
            cv2.drawChessboardCorners(img, (9,6), corners, ret)
            write_name = './camera_cal/corners_found'+str(idx)+'.jpg'
            cv2.imwrite(write_name, img)
            #cv2.imshow('img', img)
            #cv2.waitKey(500)

            # Do camera calibration given object points and image points
            img_size = (img.shape[1], img.shape[0])
            ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size, None, None)
            
            return ret, mtx, dist, rvect, tvecs

def distortion_correction(image_data, mtx, dist):
    img = np.copy(image_data)
    dst = cv2.undistort(img, mtx, dist, None, mtx)
    cv2.imwrite('./camera_cal/undist'+str(idx)+'.jpg', dst)

for idx, fname in tqdm(enumerate(images)):
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# Test undistortion on an image
#img = cv2.imread('./camera_cal/calibration1.jpg')
#img_size = (img.shape[1], img.shape[0])
        

# Save the camera calibration result for later use (we won't worry about rvecs / tvecs)
dist_pickle = {}
dist_pickle["mtx"] = mtx
dist_pickle["dist"] = dist
#print(dist_pickle)
pickle.dump( dist_pickle, open( "./camera_cal/calibration_ pickle.p", "wb" ) )

20it [00:09,  1.47it/s]


# Perspective Transform

In [ ]:
def perspective_transform(image_data):
    img = np.copy(image_data)
    # work on defining perspective transformation area
    img_size = (img.shape[1], img.shape[0])
    bot_width = 0.45#0.4#0.8#0.76 # percent of bottom trapezoid height
    mid_width = 0.12#0.04#0.2#0.08 # percent of middle trapezoid height
    height_pct = 0.66#0.66#0.62 # percent for trapezoid height
    bottom_trim = 0.935#0.935 # percent from top to bottom to avoid car hood

    src = np.float32([[img.shape[1]*(0.5-mid_width/2), img.shape[0]*height_pct], [img.shape[1]*(0.5+mid_width/2), img.shape[0]*height_pct], 
                    [img.shape[1]*(0.5+bot_width/2), img.shape[0]*bottom_trim], [img.shape[1]*(0.5-bot_width/2),  img.shape[0]*bottom_trim]])
    
    offset = img_size[0]*0.25#0.3#0.15#0.25
    dst = np.float32([[offset, 0], [img_size[0]-offset, 0], [img_size[0]-offset, img_size[1]], [offset, img_size[1]]])
    
    # perform the transform
    M = cv2.getPerspectiveTransform(src, dst)
    Minv = cv2.getPerspectiveTransform(dst,src)
    warped = cv2.warpPerspective(preprocessed_img, M, img_size, flags=cv2.INTER_LINEAR)
    
    write_name = './test_images/2. warped' + str(idx+1) + '.jpg'
    cv2.imwrite(write_name, warped)